In [3]:
#needed imports from Pandas, Numpy, Pickle, Matplot, Keras
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from os import listdir
from keras.preprocessing import sequence
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
import pprint
import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'

if os.path.exists("best_model.pkl"): os.remove("best_model.pkl")

In [4]:
#loading raw data
path = 'MovementAAL/new_dataset/stream_0000'
sequences = list()

for i in range (1,9):
    file_path = path + str(i) + '.csv'
    df = pd.read_csv(file_path, header=0, usecols=["time", "delta0", "delta1", "delta2", "delta3"])
    values = df.values
    sequences.append(values)

path = 'MovementAAL/new_dataset/stream_000'

for i in range(10, 299):
    file_path = path + str(i) + '.csv'
    df = pd.read_csv(file_path, header=0, usecols=["time", "delta0", "delta1", "delta2", "delta3"])
    sequences.append(values)


FileNotFoundError: [Errno 2] No such file or directory: 'MovementAAL/new_dataset/stream_00001.csv'

In [5]:
#target values
targets = pd.read_csv('MovementAAL/new_dataset/MovementAAL_target.csv')
targets = targets.values[:,1]
print(targets)

FileNotFoundError: [Errno 2] No such file or directory: 'MovementAAL/new_dataset/MovementAAL_target.csv'

In [6]:
#Loading grouping
groups = pd.read_csv('MovementAAL/newgroups/MovementAAL_DatasetGroup.csv', header=0)
groups = groups.values[:,1]
print(groups)

FileNotFoundError: [Errno 2] No such file or directory: 'MovementAAL/newgroups/MovementAAL_DatasetGroup.csv'

In [7]:
len_sequences = []
for one_seq in sequences:
    len_sequences.append(len(one_seq))
print(pd.Series(len_sequences).describe())

count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
dtype: float64


/opt/anaconda3/envs/project_28/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  after removing the cwd from sys.path.


In [8]:
#Padding the sequence with the values in last row to max length
to_pad = 30
new_seq = []
for one_seq in sequences:
    len_one_seq = len(one_seq)
    last_val = one_seq[-1]
    n = to_pad - len_one_seq
   

    to_concat = np.repeat(one_seq[-1], n).reshape(5, n).transpose()
    new_one_seq = np.concatenate([one_seq, to_concat])
    new_seq.append(new_one_seq)
final_seq = np.stack(new_seq)
print(final_seq)

ValueError: need at least one array to stack

In [9]:
#truncate the sequence to length 60
seq_len = 75
final_seq=sequence.pad_sequences(final_seq, maxlen=seq_len, padding='post', dtype='float', truncating='post')
print(final_seq)

NameError: name 'final_seq' is not defined

In [10]:
#Training data based on group 2
train = [final_seq[i] for i in range(len(groups)) if (groups[i]==2)]
#validation data based on group 1
validation = [final_seq[i] for i in range(len(groups)) if groups[i]==1]
#test data based on group 3
test = [final_seq[i] for i in range(len(groups)-1) if groups[i]==3]

#train target based on group 2
train_target = [targets[i] for i in range(len(groups)) if (groups[i]==2)]
#validation target based on group 1
validation_target = [targets[i] for i in range(len(groups)) if groups[i]==1]
#test target based on group 3
test_target = [targets[i] for i in range(len(groups)) if groups[i]==3]

print(train)

NameError: name 'groups' is not defined

In [11]:
#creating np.arrays for each dataset
train = np.array(train)
validation = np.array(validation)
test = np.array(test)

#training target data
train_target = np.array(train_target)
train_target = (train_target+1)/2

#validation target
validation_target = np.array(validation_target)
validation_target = (validation_target+1)/2

#test data and target test
test_target = np.array(test_target)
test_target = (test_target+1)/2

NameError: name 'train' is not defined

In [12]:
#adding the LSTM to the model and printing the summary
model = Sequential()
model.add(LSTM(120, input_shape=(seq_len, 5)))
model.add(Dense(1, activation='sigmoid'))
model.summary()
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 120)               60480     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 121       
Total params: 60,601
Trainable params: 60,601
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 120)               60480     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 121       
Total params: 60,601
Trainable params: 60,601
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
adam = Adam(lr=0.001)
chk = ModelCheckpoint('best_model.pkl', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.fit(train, train_target, epochs=200, batch_size=58, callbacks=[chk], validation_data=(validation,validation_target))


NameError: name 'train' is not defined

In [21]:
#loading the exported pkl model and testing the accuracy score
model = load_model('best_model.pkl')
from sklearn.metrics import accuracy_score
test_preds = model.predict_classes(test)
print(test)
print(test_preds)
print(test_target)
accuracy_score(test_target, test_preds)
print(accuracy_score(test_target, test_preds))

[[[ 25.  25.  51.  79.  73.]
  [ 30.  28.  62. 100.  83.]
  [ 71.  72. 105. 158. 139.]
  ...
  [  0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.]]

 [[ 25.  25.  51.  79.  73.]
  [ 30.  28.  62. 100.  83.]
  [ 71.  72. 105. 158. 139.]
  ...
  [  0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.]]

 [[ 25.  25.  51.  79.  73.]
  [ 30.  28.  62. 100.  83.]
  [ 71.  72. 105. 158. 139.]
  ...
  [  0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.]]

 ...

 [[ 25.  25.  51.  79.  73.]
  [ 30.  28.  62. 100.  83.]
  [ 71.  72. 105. 158. 139.]
  ...
  [  0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.]]

 [[ 25.  25.  51.  79.  73.]
  [ 30.  28.  62. 100.  83.]
  [ 71.  72. 105. 158. 139.]
  ...
  [  0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.]]

 [[ 25.  25.  51.  79.  73.]
  [ 30.  28.  62. 100.  83.]
  [ 71.  72. 105. 158. 139

ValueError: Found input variables with inconsistent numbers of samples: [18, 17]